# Play with Orkhut graph using evcxr_jupyter and plotters

The purpose of this notebook is to see how the communities of the orkhut graph are situated with respect to the
block decomposition


## We reload the density decompositon and the graph. 
**Code copy/pasted from graphemded/examples/orkut.rs**

In [74]:
:dep graphembed = {path = "/home/jpboth/Rust/graphembed", features=["openblas-system"]}
:dep indxvec

:dep anyhow

extern crate petgraph;
extern crate log;

use graphembed::prelude::*;
use petgraph::prelude::*;

use petgraph::{Graph, Undirected};

use std::path::{Path};

pub fn read_orkutdir(dirpath : &Path) -> Result<Graph<u32, f64 , Undirected, u32>, anyhow::Error> {
    let fpath = dirpath.clone().join("com-orkut.ungraph.txt");
    // use csv to unweighted graph_map
    log::info!("read_orkutdir : reading {fpath:?}");
    let graphmap = weighted_csv_to_graphmap::<u32 ,f64, Undirected>(&fpath, b'\t');
    log::info!("read_orkutdir : reading {fpath:?}, done");
    if graphmap.is_err() {
        std::panic!("cannot open file : {fpath:?}");
    }
    let graph = graphmap.unwrap().into_graph::<u32>();
    log::info!("graph loaded");
    Ok(graph)
} // end of read_orkutdirq

let res = StableDecomposition::reload_json(Path::new("/home/jpboth/Rust/graphembed/orkut-decomposition.json"));

let stable = res.unwrap();

let b0 = stable.get_block_points(0).unwrap();
println!("b0.len = {}", b0.len()); 

b0.len = 28289


In [91]:
let mut b0s = b0.clone();
b0s.sort();

In [93]:
b0s[0..5]

[52, 471, 535, 664, 692]

In [94]:
b0s[0..5]

[52, 471, 535, 664, 692]

#### load the graph

In [6]:
let graph = read_orkutdir(Path::new("/home/jpboth/Data/Graphs/Orkut")).unwrap();


In [88]:
pub fn get_degree_undirected<N,F>(graph : &Graph<N, F, Undirected>, rank : usize) -> Result<usize, anyhow::Error> {
    let nb_nodes = graph.node_count();
    if rank >= nb_nodes {
        return Err(anyhow::anyhow!("bad index, nb_nodes : {nb_nodes}"));
    }
    let neighbours = graph.neighbors(NodeIndex::new(rank));
    Ok(neighbours.count())
}




In [90]:
let degree_384225 = get_degree_undirected(&graph, 384225).unwrap();

In [10]:
degree_384225

1316

In [11]:
let degree_2357582 = get_degree_undirected(&graph, 2357582).unwrap();


In [12]:
degree_2357582

771

In [13]:
let mut b0mut = b0.clone();


In [14]:
:dep indxvec

In [15]:
use indxvec::Vecops;

In [16]:
let mut degrees_b0 : Vec<usize> = (0..b0.len()).into_iter().map(|i| get_degree_undirected(&graph, b0[i]).unwrap()).collect();


In [17]:
let mean_deg_b0 : f64 = degrees_b0.iter().sum::<usize>() as f64 / degrees_b0.len() as f64;


In [18]:
mean_deg_b0

942.6188978048004

**We get an index to examine degrees of bloc 0**

In [19]:
let degrees_b0_idx = degrees_b0.mergesort_indexed();
    

In [20]:
degrees_b0[degrees_b0_idx[0]]

560

In [21]:
degrees_b0[degrees_b0_idx[b0.len()-1]]

33313

In [22]:
degrees_b0[degrees_b0_idx[5]]

560

In [23]:
degrees_b0[degrees_b0_idx[500]]

564

In [24]:
let b0_sorted = b0.clone().sort();

In [25]:
stable.get_densest_block(72).unwrap()

107

#### add code to read the first 5000 communities. code copy/pasted from graphembed/examples/orkut.rs

In [27]:
use anyhow::{anyhow};

use std::io::{BufReader};
use std::fs::{OpenOptions};
use std::io::prelude::*;
use std::str::FromStr;

const ORKUT_DATA_DIR : &'static str = "/home/jpboth/Data/Graphs/Orkut/";

fn read_orkut_com(dirpath : &Path) -> anyhow::Result<Vec<Vec<usize>>> {
    let fpath = dirpath.clone().join("com-orkut.top5000.cmty.txt");
    log::info!("read_orkut_com : reading {fpath:?}");
    let fileres = OpenOptions::new().read(true).open(&fpath);
    if fileres.is_err() {
        log::error!("read_orkut_com : reload could not open file {:?}", fpath.as_os_str());
        println!("read_orkut_com could not open file {:?}", fpath.as_os_str());
        return Err(anyhow!("read_orkut_com could not open file {}", fpath.display()));            
    }
    let file = fileres?;
    let bufreader = BufReader::new(file);
    let lines = bufreader.lines();
    let mut numline = 0;
    let mut communities = Vec::<Vec<usize>>::with_capacity(5000);
    for line in lines {
        if line.is_err() {
            log::error!("error reading file : {:?} at line : {}",fpath.as_os_str(),numline);
            return Err(anyhow!(" error reading file : {:?} at line : {}",fpath.as_os_str(),numline));
        }
        // split and decode line. line consists in usize separated by a tab
        let line = line.unwrap();
        let splitted : Vec<&str>= line.split('\t').collect();
        let communitiy : Vec<usize> = splitted.iter().map(|s| usize::from_str(*s).unwrap()).collect();
        communities.push(communitiy);
        numline += 1;
    }
    //
    return Ok(communities);
} // end of read_orkut_com



In [28]:
let communities = read_orkut_com(Path::new(ORKUT_DATA_DIR)).unwrap();

In [29]:
communities.len()

5000

The first commuity has 4249 nodes

In [31]:
communities[0].len()

4249

**Examine first community**

In [33]:
let mut bc0 = communities[0].clone();

In [34]:
bc0.sort()

()

In [35]:
bc0[0..25]

[2, 5, 6, 37, 38, 41, 44, 47, 53, 56, 62, 67, 72, 73, 74, 80, 81, 173, 175, 177, 180, 309, 314, 483, 619]

In [36]:
stable.get_densest_block(74).unwrap()

191

In [50]:
let degree_73 = get_degree_undirected(&graph, 73).unwrap();
degree_73

211

In [40]:
get_degree_undirected(&graph, 74).unwrap()

18

In [49]:
stable.get_densest_block(619).unwrap()

185

**get blocks of first community**

In [42]:
let mut c0blocks = stable.get_blocks(&communities[0]);

In [43]:
c0blocks.sort();

In [44]:
c0blocks[0..25]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

**compute the mean block of community 0**

In [46]:
let meanc0 = c0blocks.iter().sum::<usize>() as f64/ c0blocks.len() as f64;
meanc0

133.2485290656625

**get list of degrees of node of first community**

In [54]:
let mut com0_degrees : Vec<usize> = communities[0].iter().map(|n| get_degree_undirected(&graph, *n).unwrap()).collect();

In [55]:
com0_degrees.sort()

()

In [56]:
com0_degrees[0..25]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [57]:
com0_degrees[2000..2025]

[63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 64]

In [58]:
com0_degrees[4000..4025]

[251, 251, 251, 251, 252, 252, 252, 253, 254, 254, 254, 255, 255, 255, 255, 258, 258, 258, 259, 259, 259, 259, 260, 260, 260]

In [59]:
let com0_mean_degree = com0_degrees.iter().sum::<usize>() as f64 / com0_degrees.len() as f64;
com0_mean_degree

95.44175100023534

#### add dependancies to get plotters

In [61]:
:dep plotters = {version = "0.3.4" ,default_features = false, features = ["evcxr", "all_series", "all_elements"] }
extern crate plotters;
use plotters::prelude::*;


histogram of blocks of community 0 (4249 nodes, blocks from 0 to 208)

## define a function displaying blocks of nodes in a communitiy

In [83]:
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper;

fn display_blocks(block_com : &Vec<usize>, stable : &StableDecomposition) -> SVGWrapper {
    
let nb_blocks = stable.get_nb_blocks();
println!(" communitity size : {}", block_com.len());
    
let fig = evcxr_figure((800, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("blocks histogram", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0usize..nb_blocks, 0f64..1.0f64)?;
    
    // configure axis
    chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .y_labels(5)
            .x_label_formatter(&|x| format!("{:.1}", *x as usize))
            .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
            .draw()?;
    
    let hist_block = Histogram::vertical(&chart)
        .style(RED.filled())
        .margin(0)
        .data(block_com.iter().map(|x| ((*x), 200./ stable.get_nbpoints_in_block(*x).unwrap() as f64)));
    
    let _res = chart.draw_series(hist_block);
    
    Ok(())
    }).style("width:80%");
    fig
}

In [84]:
let mut c0blocks = stable.get_blocks(&communities[0]);
display_blocks(&c0blocks, &stable)

 communitity size : 4249


blocks histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200

In [85]:
let mut c1blocks = stable.get_blocks(&communities[1]);
display_blocks(&c1blocks, &stable)

 communitity size : 837


blocks histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200

In [86]:
let mut c22blocks = stable.get_blocks(&communities[22]);
display_blocks(&c22blocks, &stable)

 communitity size : 4785


blocks histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200

In [87]:
let mut c4blocks = stable.get_blocks(&communities[4]);
display_blocks(&c4blocks, &stable)

 communitity size : 55


blocks histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200

**It seems there is a node in block 0. This small community has a member in the highest density block**

In [95]:
c4blocks.sort()

()

In [97]:
c4blocks[0..5]

[0, 73, 73, 79, 79]

In [98]:
let mut c6blocks = stable.get_blocks(&communities[6]);
display_blocks(&c6blocks, &stable)

 communitity size : 3


blocks histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200

In [99]:
communities[6]

[960274, 960277, 960279]